# SQL Notebook for SQL Server 2022 data virtualization for delta tables on S3 object storage

This is a SQL notebook to show the data virtualization capabilities in SQL Server 2022 for delta tables on S3 object storage. You can find all the scripts and files referenced in this notebook at [https://aka.ms/sqlserver2022demos](https://aka.ms/sqlserver2022demos) under the datavirt/delta folder

# Prerequisites for the exercise

- SQL Server 2022 Evaluation Edition with the Database Engine and PolyBase Query Service for External Data Feature installed. You can use the defaults in setup for Polybase.
- VM or computer with 2 CPUs and at least 8Gb RAM.
- SQL Server Management Studio (SSMS). The latest 18.x build or 19.x build will work.

    **Note**: The following pre-requisites are for non-Microsoft software. The use of this software does not represent any official endorsement from Microsoft. This software is not supported by Microsoft so any issues using this software are up to the user to resolve.

- The **minio** server for Windows which you can download at https://min.io/download#/windows. For the demo I assume you have created a directory called c:\minio and have downloaded the minio.exe for Windows into that directory.
- openssl for Windows which you can download at https://slproweb.com/products/Win32OpenSSL.html. I chose the Win64 OpenSSL v3.0.5 MSI option.

# Steps to setup minio for the exercise

- Download minio.exe for Windows into c:\minio.exe
- Download the openssl for Windows MSI and run the installer. Use all the defaults.
- Set a system environment variable OPENSSL_CONF=C:\Program Files\OpenSSL-Win64\bin\openssl.cfg and add to the system environment variable PATH c:\Program Files\OpenSSL-Win64\bin
- Generate a private key using the following command from the c:\minio directory.

    `openssl genrsa -out private.key 2048`

- Copy the supplied openssl.conf to c:\minio. Edit openssl.conf by changing IP.2 to your local IP address and DNS.2 to your local computer name.
- From the c:\minio directory run the following command to generate a self-signed certificate

    `openssl req -new -x509 -nodes -days 730 -key private.key -out public.crt -config openssl.conf`

- Double-click the public.crt file and select Install Certificate. Choose Local Machine, Place all certificates in the following store, Browse, and select Trusted Root Certification Authorities.
- Copy the private.key and public.crt files from c:\minio into %%USERPROFILE%%\.minio\certs.

# Steps to use mino for the exercise

From a command prompt at the c:\\minio directory start the minio server with the following command (example syntax for Powershell)

`.\minio.exe server c:\minio --console-address ":9001"`

This program will startup and "run forever" until stopped with +. Your output should look similar to the following (the IP addresses will match your local IP):

```
MinIO Object Storage Server
Copyright: 2015-2022 MinIO, Inc.
License: GNU AGPLv3 <https://www.gnu.org/licenses/agpl-3.0.html>
Version: RELEASE.2022-07-30T05-21-40Z (go1.18.4 windows/amd64)

Status:         1 Online, 0 Offline.
API: https://<local IP>:9000  https://127.0.0.1:9000
RootUser: <user>
RootPass: <password>
Console: https://<local IP>:9001 https://127.0.0.1:9001
RootUser: <user>
RootPass: <password>

Command-line: https://docs.min.io/docs/minio-client-quickstart-guide
   $ mc.exe alias set myminio https://<local IP>:9000 <user> <password>
``
Documentation: https://docs.min.io

```

1. Test your connection and browse the minio storage system using a web browser. Use the address [https://127.0.0.1:9001](https://127.0.0.1:9001/). You should be presented with a login screen. Use the defaut root user and password which are displayed on the minio server screen.
    
2. On the left-hand side menu, click on Identity and Users. Select Create User. Create a user name with password. Select the readwrite policy for the user. This is the user and password you will use for the SECRET value in creates3creds.sql.
    
3. Select menu for Buckets. Select Create Bucket. Use a Bucket Name of **backups**. Leave all defaults and select Create Bucket.

# Steps to use SQL Server 2022 for the exercise

Run each cell one at time to use SQL Server 2022 for the exercise

## Step 1 - Get WideWorldImporters

Copy the **WideWorldImporters** sample database from [https://aka.ms/WideWorldImporters](https://aka.ms/WideWorldImporters) to a local directory (The restore script assumes **c:\\sql\_sample\_databases**)

## Step 2 - Restore WideWorldImporters

Edit the following T-SQL to restore the WideWorldImporters database for your correct files paths and execute the statement

In [4]:
USE master;
GO
DROP DATABASE IF EXISTS WideWorldImporters;
GO
RESTORE DATABASE WideWorldImporters FROM DISK = 'c:\sql_sample_databases\WideWorldImporters-Full.bak' with
MOVE 'WWI_Primary' TO 'c:\sql_sample_databases\WideWorldImporters.mdf',
MOVE 'WWI_UserData' TO 'c:\sql_sample_databases\WideWorldImporters_UserData.ndf',
MOVE 'WWI_Log' TO 'c:\sql_sample_databases\WideWorldImporters.ldf',
MOVE 'WWI_InMemory_Data_1' TO 'c:\sql_sample_databases\WideWorldImporters_InMemory_Data_1',
stats=5;
GO
ALTER DATABASE WideWorldImporters SET QUERY_STORE CLEAR ALL;
GO

Commands completed successfully.

Commands completed successfully.

5 percent processed.

10 percent processed.

15 percent processed.

...

Database 'WideWorldImporters' running the upgrade step from version 954 to version 955.

100 percent processed.

RESTORE DATABASE successfully processed 58455 pages in 1.801 seconds (253.567 MB/sec).

Commands completed successfully.

Total execution time: 00:00:07.326

# Step 3 - Storage credential

Create a credential against the s3 storage target

In [2]:
USE MASTER
GO
IF EXISTS (SELECT * FROM sys.credentials WHERE name = 's3://<local IP>/backups')
	DROP CREDENTIAL [s3://<local IP>:9000/backups];
GO
CREATE CREDENTIAL [s3://<local IP>:9000/backups]
WITH IDENTITY = 'S3 Access Key',
SECRET = '<user>:<password>';
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.049

# Step 4 - Backup to S3

Execute various types of backups against S3

In [5]:
USE MASTER;
GO
ALTER DATABASE WideWorldImporters SET RECOVERY FULL;
GO
BACKUP DATABASE WideWorldImporters
TO URL = 's3://<local IP>:9000/backups/wwi.bak'
WITH CHECKSUM, INIT, FORMAT;
GO
BACKUP DATABASE WideWorldImporters
TO URL = 's3://<local IP>:9000/backups/wwidiff.bak'
WITH CHECKSUM, INIT, FORMAT, DIFFERENTIAL
GO
BACKUP LOG WideWorldImporters
TO URL = 's3://<local IP>:9000/backups/wwilog.bak'
WITH CHECKSUM, INIT, FORMAT
GO
BACKUP DATABASE WideWorldImporters
FILE = 'WWI_UserData'
TO URL = 's3://<local IP>:9000/backups/wwiuserdatafile.bak'
WITH CHECKSUM, INIT, FORMAT;
GO

Commands completed successfully.

Commands completed successfully.

Processed 1328 pages for database 'WideWorldImporters', file 'WWI_Primary' on file 1.

Processed 53104 pages for database 'WideWorldImporters', file 'WWI_UserData' on file 1.

Processed 0 pages for database 'WideWorldImporters', file 'WWI_InMemory_Data_1' on file 1.

...

BACKUP LOG successfully processed 61 pages in 0.177 seconds (2.670 MB/sec).

Processed 53096 pages for database 'WideWorldImporters', file 'WWI_UserData' on file 1.

Processed 689 pages for database 'WideWorldImporters', file 'WWI_Log' on file 1.

BACKUP DATABASE...FILE= successfully processed 53785 pages in 6.152 seconds (68.301 MB/sec).

Total execution time: 00:00:14.353

# Step 5 - Browse backup files in S3

Use the minio console to browse the backup files created in the backups bucket in s3

# Step 6 - Restore the backup

Executed the following RESTORE options to look at the full backup file and restore the backup to a new database name

In [6]:
USE MASTER;
GO
RESTORE VERIFYONLY FROM URL = 's3://<local IP>:9000/backups/wwi.bak';
GO
RESTORE HEADERONLY FROM URL = 's3://<local IP>:9000/backups/wwi.bak';
GO
RESTORE FILELISTONLY FROM URL = 's3://<local IP>:9000/backups/wwi.bak';
GO
DROP DATABASE IF EXISTS WideWorldImporters2;
GO
RESTORE DATABASE WideWorldImporters2 
FROM URL = 's3://<local IP>:9000/backups/wwi.bak'
WITH MOVE 'WWI_Primary' TO 'c:\sql_sample_databases\WideWorldImporters2.mdf',
MOVE 'WWI_UserData' TO 'c:\sql_sample_databases\WideWorldImporters2_UserData.ndf',
MOVE 'WWI_Log' TO 'c:\sql_sample_databases\WideWorldImporters2.ldf',
MOVE 'WWI_InMemory_Data_1' TO 'c:\sql_sample_databases\WideWorldImporters2_InMemory_Data_1';
GO

Commands completed successfully.

The backup set on file 1 is valid.

(1 row affected)

(4 rows affected)

Commands completed successfully.

...

Processed 3865 pages for database 'WideWorldImporters2', file 'WWI_InMemory_Data_1' on file 1.

RESTORE DATABASE successfully processed 58974 pages in 1.696 seconds (271.657 MB/sec).

Total execution time: 00:00:07.553

BackupName,BackupDescription,BackupType,ExpirationDate,Compressed,Position,DeviceType,UserName,ServerName,DatabaseName,DatabaseVersion,DatabaseCreationDate,BackupSize,FirstLSN,LastLSN,CheckpointLSN,DatabaseBackupLSN,BackupStartDate,BackupFinishDate,SortOrder,CodePage,UnicodeLocaleId,UnicodeComparisonStyle,CompatibilityLevel,SoftwareVendorId,SoftwareVersionMajor,SoftwareVersionMinor,SoftwareVersionBuild,MachineName,Flags,BindingID,RecoveryForkID,Collation,FamilyGUID,HasBulkLoggedData,IsSnapshot,IsReadOnly,IsSingleUser,HasBackupChecksums,IsDamaged,BeginsLogChain,HasIncompleteMetaData,IsForceOffline,IsCopyOnly,FirstRecoveryForkID,ForkPointLSN,RecoveryModel,DifferentialBaseLSN,DifferentialBaseGUID,BackupTypeDescription,BackupSetGUID,CompressedBackupSize,Containment,KeyAlgorithm,EncryptorThumbprint,EncryptorType,LastValidRestoreTime,TimeZone,CompressionAlgorithm
NULL,NULL,1,NULL,0,1,9,BW-SQL2022\Administrator,bw-sql2022,WideWorldImporters,955,2022-08-03 19:50:39.000,519838720,626000002505600002,627000001042400001,627000001040000001,627000001012800001,2022-08-04 17:10:51.000,2022-08-04 17:10:57.000,0,0,1033,196609,130,4608,16,0,700,BW-SQL2022,528,e373cfd3-775b-489e-a3ad-9dd737e1c86a,918cdbdd-e60c-4db5-b57e-bab507756b31,Latin1_General_100_CI_AS,1cc0a00a-1361-463f-b195-df7642a9754e,0,0,0,0,1,0,0,0,0,0,918cdbdd-e60c-4db5-b57e-bab507756b31,NULL,FULL,NULL,NULL,Database,72ed30fa-af45-469f-91a6-b218c6fddbe5,519838720,0,NULL,NULL,NULL,NULL,1,NULL


LogicalName,PhysicalName,Type,FileGroupName,Size,MaxSize,FileId,CreateLSN,DropLSN,UniqueId,ReadOnlyLSN,ReadWriteLSN,BackupSizeInBytes,SourceBlockSize,FileGroupId,LogGroupGUID,DifferentialBaseLSN,DifferentialBaseGUID,IsReadOnly,IsPresent,TDEThumbprint,SnapshotUrl
WWI_Primary,c:\sql_sample_databases\WideWorldImporters.mdf,D,PRIMARY,1073741824,35184372080640,1,0,0,8d30f4f9-a463-404f-805a-9bd1c634b66b,0,0,10747904,4096,1,NULL,627000001012800001,51e9fd64-0ed0-480b-98a2-e9c5a36280b9,0,1,NULL,NULL
WWI_UserData,c:\sql_sample_databases\WideWorldImporters_UserData.ndf,D,USERDATA,2147483648,35184372080640,3,37000000095200001,0,28d406e0-78ff-4400-9a4b-3a05d136b1f3,0,0,434962432,4096,2,NULL,627000001028000034,75e38a5c-9adb-4f7d-9f93-2d4cd9a71c7b,0,1,NULL,NULL
WWI_Log,c:\sql_sample_databases\WideWorldImporters.ldf,L,NULL,104857600,2199023255552,2,0,0,6ac6807e-8774-415b-8efc-e8c569b0855e,0,0,0,4096,0,NULL,0,00000000-0000-0000-0000-000000000000,0,1,NULL,NULL
WWI_InMemory_Data_1,c:\sql_sample_databases\WideWorldImporters_InMemory_Data_1,S,WWI_InMemory_Data,0,0,65537,624000000336200003,0,f65663c8-a250-433e-bbe6-e13a5599a607,0,0,200671232,4096,3,NULL,627000001012800001,51e9fd64-0ed0-480b-98a2-e9c5a36280b9,0,1,NULL,NULL


# Step 7 - Browse backup and restore metadata

Browse the standard msdb system tables such as **backupset,** **backupmediafamily**, and **restorehistory**